In [2]:
import tkinter as tk
from tkinter import messagebox
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

def connect_db():
    try:
        conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(
            conexao_bd,
            connect_args={'options': '-csearch_path=sistema,public'}
        )
        conn = engine.connect()
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

def insert_cliente(nome, cpf, emails, telefones, rua, numero, bairro, cidade, estado, convenio, alergias):
    conn = connect_db()  # Mantendo a função de conexão com o banco
    if conn:
        try:
            # Início da transação
            trans = conn.begin()

            # Inserir na tabela Pessoa
            pessoa_query = text("INSERT INTO Pessoa (Nome, CPF) VALUES (:nome, :cpf) RETURNING ID_Pessoa")
            pessoa_result = conn.execute(pessoa_query, {"nome": nome, "cpf": cpf})
            id_pessoa = pessoa_result.fetchone()[0]
            print(f"ID_Pessoa: {id_pessoa}")
            
            # Inserir múltiplos emails
            email_query = text("INSERT INTO Email (Email, ID_Pessoa) VALUES (:email, :id_pessoa)")
            for email in emails.split(','):
                conn.execute(email_query, {"email": email.strip(), "id_pessoa": id_pessoa})
            
            # Inserir múltiplos telefones
            telefone_query = text("INSERT INTO Telefone (Telefone, ID_Pessoa) VALUES (:telefone, :id_pessoa)")
            for telefone in telefones.split(','):
                conn.execute(telefone_query, {"telefone": telefone.strip(), "id_pessoa": id_pessoa})
            
            # Inserir na tabela Endereco
            endereco_query = text("INSERT INTO Endereco (Rua, Numero, Bairro, Cidade, Estado, ID_Pessoa) VALUES (:rua, :numero, :bairro, :cidade, :estado, :id_pessoa)")
            conn.execute(endereco_query, {"rua": rua, "numero": numero, "bairro": bairro, "cidade": cidade, "estado": estado, "id_pessoa": id_pessoa})
            
            # Inserir na tabela Cliente
            cliente_query = text("INSERT INTO Cliente (Convenio, ID_Pessoa) VALUES (:convenio, :id_pessoa) RETURNING ID_Cliente")
            cliente_result = conn.execute(cliente_query, {"convenio": convenio, "id_pessoa": id_pessoa})
            id_cliente = cliente_result.fetchone()[0]
            print(f"ID_Cliente: {id_cliente}")
            
            # Inserir alergias do cliente
            alergia_query = text("INSERT INTO Alergia (Descricao, ID_Cliente) VALUES (:descricao, :id_cliente)")
            for alergia in alergias.split(','):
                conn.execute(alergia_query, {"descricao": alergia.strip(), "id_cliente": id_cliente})

            # Commit da transação
            trans.commit()
            messagebox.showinfo("Sucesso", "Cliente cadastrado com sucesso!")
        except Exception as e:
            trans.rollback()
            messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o cliente: {e}")
            print(f"Erro ao cadastrar cliente: {str(e)}")
        finally:
            conn.close()

# Função para capturar os dados da interface e chamar a função de inserção
def salvar_cliente():
    nome = entry_nome.get()
    cpf = entry_cpf.get()
    emails = entry_email.get()  # Suporta múltiplos emails separados por vírgula
    telefones = entry_telefone.get()  # Suporta múltiplos telefones separados por vírgula
    rua = entry_rua.get()
    numero = entry_numero.get()
    bairro = entry_bairro.get()
    cidade = entry_cidade.get()
    estado = entry_estado.get()
    convenio = entry_convenio.get()  # Convenio do cliente
    alergias = entry_alergias.get()  # Alergias do cliente

    # Inserir os dados no banco
    insert_cliente(nome, cpf, emails, telefones, rua, numero, bairro, cidade, estado, convenio, alergias)

# Interface gráfica usando Tkinter
root = tk.Tk()
root.title("Cadastro de Cliente")

# Labels e campos de texto para as informações do cliente
tk.Label(root, text="Nome").grid(row=0, column=0)
entry_nome = tk.Entry(root)
entry_nome.grid(row=0, column=1)

tk.Label(root, text="CPF").grid(row=1, column=0)
entry_cpf = tk.Entry(root)
entry_cpf.grid(row=1, column=1)

tk.Label(root, text="Email").grid(row=2, column=0)
entry_email = tk.Entry(root)
entry_email.grid(row=2, column=1)

tk.Label(root, text="Telefone").grid(row=3, column=0)
entry_telefone = tk.Entry(root)
entry_telefone.grid(row=3, column=1)

tk.Label(root, text="Rua").grid(row=4, column=0)
entry_rua = tk.Entry(root)
entry_rua.grid(row=4, column=1)

tk.Label(root, text="Número").grid(row=5, column=0)
entry_numero = tk.Entry(root)
entry_numero.grid(row=5, column=1)

tk.Label(root, text="Bairro").grid(row=6, column=0)
entry_bairro = tk.Entry(root)
entry_bairro.grid(row=6, column=1)

tk.Label(root, text="Cidade").grid(row=7, column=0)
entry_cidade = tk.Entry(root)
entry_cidade.grid(row=7, column=1)

tk.Label(root, text="Estado").grid(row=8, column=0)
entry_estado = tk.Entry(root)
entry_estado.grid(row=8, column=1)

# Campo para convênio
tk.Label(root, text="Convênio").grid(row=9, column=0)
entry_convenio = tk.Entry(root)
entry_convenio.grid(row=9, column=1)

# Campo para alergias
tk.Label(root, text="Alergias").grid(row=10, column=0)
entry_alergias = tk.Entry(root)
entry_alergias.grid(row=10, column=1)

# Botão de salvar
btn_salvar = tk.Button(root, text="Salvar", command=salvar_cliente)
btn_salvar.grid(row=11, column=1)

# Loop da interface gráfica
root.mainloop()


ID_Pessoa: 27
ID_Cliente: 5
Erro ao cadastrar cliente: (psycopg2.errors.UniqueViolation) ERRO:  duplicar valor da chave viola a restrição de unicidade "pessoa_cpf_key"
DETAIL:  Chave (cpf)=(02154578995) já existe.

[SQL: INSERT INTO Pessoa (Nome, CPF) VALUES (%(nome)s, %(cpf)s) RETURNING ID_Pessoa]
[parameters: {'nome': 'Ana Luiza Piancó', 'cpf': '02154578995'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
ID_Pessoa: 29
ID_Cliente: 6
Erro ao cadastrar cliente: (psycopg2.errors.UniqueViolation) ERRO:  duplicar valor da chave viola a restrição de unicidade "pessoa_cpf_key"
DETAIL:  Chave (cpf)=(02154578994) já existe.

[SQL: INSERT INTO Pessoa (Nome, CPF) VALUES (%(nome)s, %(cpf)s) RETURNING ID_Pessoa]
[parameters: {'nome': 'Ana Luiza Piancoo', 'cpf': '02154578994'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
